In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp optimization

# Optimization
Utilities for hyperparameter optimization.

In [ ]:
#| export
import copy
from typing import Any, Callable, Dict, Optional, Union

import numpy as np
import optuna
import utilsforecast.processing as ufp
from sklearn.base import BaseEstimator, clone
from utilsforecast.compat import DataFrame

from mlforecast import MLForecast
from mlforecast.compat import CatBoostRegressor
from mlforecast.core import Freq

In [ ]:
#| exporti
_TrialToConfig = Callable[[optuna.Trial], Dict[str, Any]]

In [ ]:
#| export
def mlforecast_objective(
    df: DataFrame,
    config_fn: _TrialToConfig,
    loss: Callable,
    model: BaseEstimator,
    freq: Freq,
    n_windows: int,
    h: int,
    step_size: Optional[int] = None,
    input_size: Optional[int] = None,
    refit: Union[bool, int] = False,
    id_col: str = 'unique_id',
    time_col: str = 'ds',
    target_col: str = 'y',
) -> Callable[[optuna.Trial], float]:
    """optuna objective function for the MLForecast class
    
    Parameters
    ----------
    config_fn : callable
        Function that takes an optuna trial and produces a configuration with the following keys:
        - model_params
        - mlf_init_params
        - mlf_fit_params
    loss : callable
        Function that takes the validation and train dataframes and produces a float.
    model : BaseEstimator
        scikit-learn compatible model to be trained
    freq : str or int
        pandas' or polars' offset alias or integer denoting the frequency of the series.
    n_windows : int
        Number of windows to evaluate.
    h : int
        Forecast horizon.
    step_size : int, optional (default=None)
        Step size between each cross validation window. If None it will be equal to `h`.
    input_size : int, optional (default=None)
        Maximum training samples per serie in each window. If None, will use an expanding window.
    refit : bool or int (default=False)
        Retrain model for each cross validation window.
        If False, the models are trained at the beginning and then used to predict each window.
        If positive int, the models are retrained every `refit` windows.        
    id_col : str (default='unique_id')
        Column that identifies each serie.
    time_col : str (default='ds')
        Column that identifies each timestep, its values can be timestamps or integers.
    target_col : str (default='y')
        Column that contains the target.        
    study_kwargs : dict, optional (default=None)
    """
    def objective(trial: optuna.Trial) -> float:
        config = config_fn(trial)
        trial.set_user_attr('config', copy.deepcopy(config))
        if all(
            config['mlf_init_params'].get(k, None) is None
            for k in ['lags', 'lag_transforms', 'date_features']
        ):
            # no features
            return np.inf
        splits = ufp.backtest_splits(
            df,
            n_windows=n_windows,
            h=h,
            id_col=id_col,
            time_col=time_col,
            freq=freq,
            step_size=step_size,
            input_size=input_size,
        )
        model_copy = clone(model)
        model_params = config['model_params']
        if (
            config['mlf_fit_params'].get('static_features', [])
            and isinstance(model, CatBoostRegressor)
        ):
            # catboost needs the categorical features in the init signature
            # we assume all statics are categoricals
            model_params['cat_features'] = config['mlf_fit_params']['static_features']
        model_copy.set_params(**config['model_params'])
        metrics = []
        mlf = MLForecast(
            models={'model': model_copy},
            freq=freq,
            **config['mlf_init_params'],
        )        
        for i, (_, train, valid) in enumerate(splits):
            should_fit = i == 0 or (refit > 0 and i % refit == 0)
            if should_fit:
                mlf.fit(
                    train,
                    id_col=id_col,
                    time_col=time_col,
                    target_col=target_col,
                    **config['mlf_fit_params'],
                )
            static = [c for c in mlf.ts.static_features_.columns if c != id_col]
            dynamic = [
                c
                for c in valid.columns
                if c not in static + [id_col, time_col, target_col]
            ]
            if dynamic:
                X_df: Optional[DataFrame] = ufp.drop_columns(
                    valid, static + [target_col]
                )
            else:
                X_df = None
            preds = mlf.predict(
                h=h,
                X_df=X_df,
                new_df=None if should_fit else train,
            )
            result = ufp.join(
                valid[[id_col, time_col, target_col]],
                preds,
                on=[id_col, time_col],
            )
            if result.shape[0] < valid.shape[0]:
                raise ValueError(
                    "Cross validation result produced less results than expected. "
                    "Please verify that the passed frequency (freq) matches your series' "
                    "and that there aren't any missing periods."                    
                )
            metric = loss(result, train_df=train)
            metrics.append(metric)
            trial.report(metric, step=i)
            if trial.should_prune():
                raise optuna.TrialPruned()
        return np.mean(metrics).item()
    return objective

In [ ]:
#| hide
from nbdev import show_doc

In [ ]:
show_doc(mlforecast_objective)

---

[source](https://github.com/Nixtla/mlforecast/blob/main/mlforecast/optimization.py#L24){target="_blank" style="float:right; font-size:smaller"}

### mlforecast_objective

>      mlforecast_objective
>                            (df:Union[pandas.core.frame.DataFrame,polars.datafr
>                            ame.frame.DataFrame], config_fn:Callable[[optuna.tr
>                            ial._trial.Trial],Dict[str,Any]], loss:Callable,
>                            model:sklearn.base.BaseEstimator,
>                            freq:Union[int,str], n_windows:int, h:int,
>                            step_size:Optional[int]=None,
>                            input_size:Optional[int]=None,
>                            refit:Union[bool,int]=False,
>                            id_col:str='unique_id', time_col:str='ds',
>                            target_col:str='y')

*optuna objective function for the MLForecast class*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  |  |
| config_fn | Callable |  | Function that takes an optuna trial and produces a configuration with the following keys:<br>- model_params<br>- mlf_init_params<br>- mlf_fit_params |
| loss | Callable |  | Function that takes the validation and train dataframes and produces a float. |
| model | BaseEstimator |  | scikit-learn compatible model to be trained |
| freq | Union |  | pandas' or polars' offset alias or integer denoting the frequency of the series. |
| n_windows | int |  | Number of windows to evaluate. |
| h | int |  | Forecast horizon. |
| step_size | Optional | None | Step size between each cross validation window. If None it will be equal to `h`. |
| input_size | Optional | None |  |
| refit | Union | False | Retrain model for each cross validation window.<br>If False, the models are trained at the beginning and then used to predict each window.<br>If positive int, the models are retrained every `refit` windows.         |
| id_col | str | unique_id | Column that identifies each serie. |
| time_col | str | ds | Column that identifies each timestep, its values can be timestamps or integers. |
| target_col | str | y | Column that contains the target.         |
| **Returns** | **Callable** |  |  |

In [ ]:
import lightgbm as lgb
from datasetsforecast.m4 import M4, M4Evaluation, M4Info
from utilsforecast.losses import smape

from mlforecast.lag_transforms import ExpandingMean, RollingMean
from mlforecast.target_transforms import Differences, LocalBoxCox, LocalStandardScaler

In [ ]:
def train_valid_split(group):
    df, *_ = M4.load(directory='data', group=group)
    df['ds'] = df['ds'].astype('int')
    horizon = M4Info[group].horizon
    valid = df.groupby('unique_id').tail(horizon)
    train = df.drop(valid.index)
    return train, valid

In [ ]:
h = M4Info['Weekly'].horizon
weekly_train, weekly_valid = train_valid_split('Weekly')
weekly_train['unique_id'] = weekly_train['unique_id'].astype('category')
weekly_valid['unique_id'] = weekly_valid['unique_id'].astype(weekly_train['unique_id'].dtype)

In [ ]:
def config_fn(trial):
    candidate_lags = [
        [1],
        [13],
        [1, 13],
        range(1, 33),
    ]
    lag_idx = trial.suggest_categorical('lag_idx', range(len(candidate_lags)))
    candidate_lag_tfms = [
        {
            1: [RollingMean(window_size=13)]
        },
        {
            1: [RollingMean(window_size=13)],
            13: [RollingMean(window_size=13)],
        },
        {
            13: [RollingMean(window_size=13)],
        },
        {
            4: [ExpandingMean(), RollingMean(window_size=4)],
            8: [ExpandingMean(), RollingMean(window_size=4)],
        }
    ]
    lag_tfms_idx = trial.suggest_categorical('lag_tfms_idx', range(len(candidate_lag_tfms)))
    candidate_targ_tfms = [
        [Differences([1])],
        [LocalBoxCox()],
        [LocalStandardScaler()],        
        [LocalBoxCox(), Differences([1])],
        [LocalBoxCox(), LocalStandardScaler()],
        [LocalBoxCox(), Differences([1]), LocalStandardScaler()],
    ]
    targ_tfms_idx = trial.suggest_categorical('targ_tfms_idx', range(len(candidate_targ_tfms)))
    return {
        'model_params': {
            'learning_rate': 0.05,
            'objective': 'l1',
            'bagging_freq': 1,
            'num_threads': 2,
            'verbose': -1,
            'force_col_wise': True,
            'n_estimators': trial.suggest_int('n_estimators', 10, 1000, log=True),            
            'num_leaves': trial.suggest_int('num_leaves', 31, 1024, log=True),
            'lambda_l1': trial.suggest_float('lambda_l1', 0.01, 10, log=True),
            'lambda_l2': trial.suggest_float('lambda_l2', 0.01, 10, log=True),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.75, 1.0),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.75, 1.0),
        },
        'mlf_init_params': {
            'lags': candidate_lags[lag_idx],
            'lag_transforms': candidate_lag_tfms[lag_tfms_idx],
            'target_transforms': candidate_targ_tfms[targ_tfms_idx],
        },
        'mlf_fit_params': {
            'static_features': ['unique_id'],
        }
    }

def loss(df, train_df):
    return smape(df, models=['model'])['model'].mean()

In [ ]:
optuna.logging.set_verbosity(optuna.logging.WARNING)
objective = mlforecast_objective(
    df=weekly_train,
    config_fn=config_fn,
    loss=loss,    
    model=lgb.LGBMRegressor(),
    freq=1,
    n_windows=2,
    h=h,
)
study = optuna.create_study(
    direction='minimize', sampler=optuna.samplers.TPESampler(seed=0)
)
study.optimize(objective, n_trials=2)
best_cfg = study.best_trial.user_attrs['config']
final_model = MLForecast(
    models=[lgb.LGBMRegressor(**best_cfg['model_params'])],
    freq=1,
    **best_cfg['mlf_init_params'],
)
final_model.fit(weekly_train, **best_cfg['mlf_fit_params'])
preds = final_model.predict(h)
M4Evaluation.evaluate('data', 'Weekly', preds['LGBMRegressor'].values.reshape(-1, 13))

,SMAPE,MASE,OWA
Weekly,9.261538,2.614473,0.976158
